In [292]:
import re
import string
from nltk.tokenize import WhitespaceTokenizer 

In [293]:
white_space_tokenizer = WhitespaceTokenizer()

In [294]:
data = 'tachthucong.txt'
file_name_dictionary = "Viet74K.txt"

In [295]:
sentences = list()

with open(data, "r", encoding='utf-8') as f:
    content = f.readlines()
    for i in range(len(content)):
        sentence = content[i]
        sentences.append(sentence)
        
print("Number of sentences in data:", len(sentences))
print(sentences)

Number of sentences in data: 40
['đến nay, lực_lượng chức_năng đã xác_định trên 600 F1 và hàng nghìn F2 liên_quan nhóm bệnh_nhân này.\n', 'dự_báo, số F0 tại xã Hải_Minh dự_kiến sẽ còn gia_tăng trong những ngày tới.\n', 'đến nay, lực_lượng chức_năng đã xác_định trên 600 F1 và hàng nghìn F2 liên_quan nhóm bệnh_nhân này.\n', 'ông Nguyễn_Ngọc_Toàn, tổng_biên_tập báo Thanh_Niên, trao thiết_bị hỗ_trợ học_tập và gửi_gắm nhiều niềm hy_vọng cho các em sẽ gặt_hái được nhiều thành_công học_tập.\n', '24 học_sinh nhận thiết_bị hỗ_trợ từ chương_trình là 24 hoàn_cảnh khó_khăn khác nhau nhưng cùng chung ước_muốn được học trực_tuyến cùng thầy_cô và bạn_bè.\n', 'hay đó là Bùi_Gia_Phúc, học_sinh lớp 4/5 trường tiểu_học Trần_Văn_Ơn, sống với ông_bà nội ngay khi mới chỉ một tháng tuổi.\n', 'thông_qua đó thêm một cơ_hội để các em vượt qua khó_khăn, thử_thách, vươn lên đạt kết_quả tốt nhất trong học_tập, đóng_góp cho tương_lai.\n', 'chương_trình sẽ là dịp để lan tỏa và san_sẻ tình_yêu_thương trong đại_dịch.\

In [296]:
dictionary_corpus = list()

with open(file_name_dictionary, "r", encoding='utf-8') as f:
    content = f.readlines()
    f.close()

    for i in range(len(content)):
        term = content[i].strip().lower()
        dictionary_corpus.append(term)
        
dictionary_corpus

print("Number of terms in dictionary:", len(dictionary_corpus))

Number of terms in dictionary: 73901


In [297]:
punch =  string.punctuation
characters_to_remove = '_'
for character in characters_to_remove:
    punch = punch.replace(character,'')

In [298]:
def PreprocessString(string_data):
    # Lower the text
    preprocess_data = string_data.lower()
    
    # Remove punctuations, each punctuation = space, ex: ""information @#$retrieval" -> "information    retrieval"
    preprocess_data = re.sub('[%s]' % re.escape(punch), ' ', preprocess_data)   
        
    # Tokenize word by white space
    preprocess_data = white_space_tokenizer.tokenize(preprocess_data)
      
    return preprocess_data

In [299]:
def GetTerm(sentence_tokenized, start_index, end_index):
    term = ""
    for i in range(start_index, end_index):
        term += sentence_tokenized[i] + " "
    
    return term.strip()

In [300]:
LENGTH_MAX_TERM_VIETNAMESE = 4
LENGTH_MAX_TERM = LENGTH_MAX_TERM_VIETNAMESE

def maximumMatching(sentences):
    sentences_tokenized = dict()

    for ith_sentence, sentence in enumerate(sentences):
        """ Hiện tại thì chỗ này xét cả câu luôn, về sau sẽ nâng cấp lên thành xét các vế trong câu """
        sentence_part_tokenzied = PreprocessString(sentence)

        ith_start_term = 0
        if len(sentence_part_tokenzied) > LENGTH_MAX_TERM:
            ith_end_term = ith_start_term + LENGTH_MAX_TERM
        else:
            ith_end_term = len(sentence_part_tokenzied)

        term = GetTerm(sentence_part_tokenzied, ith_start_term, ith_end_term)

        # Xét hết cả vế
        while True:

            # Xét hết cả term
            while ith_start_term != ith_end_term:
                if term in dictionary_corpus:

                    if ith_sentence not in sentences_tokenized.keys():
                        sentences_tokenized[ith_sentence] = [term]
                    else:
                        sentences_tokenized[ith_sentence].append(term)
    #                 print("Trùng được term:", term)

                    break
                else:
    #                 print("không trùng term nào cả, term:", term)
                    ith_end_term -= 1
                    term = GetTerm(sentence_part_tokenzied, ith_start_term, ith_end_term)

            # Đã xét xong term cuối cùng thì chuyển sang câu (vế) khác
            if ith_end_term == len(sentence_part_tokenzied):
                break
            # Nếu vẫn còn term để xét thì chuyển sang term kế tiếp
            else:
                # Đề phòng trường hợp term không có trong dictionary thì end sẽ tiến về và bằng start, và sẽ gây ra vòng lặp vô tận
                # nếu cứ gắn start bằng lại end (giá trị start ban đầu) và cộng end = start + LEN_MAX_TERM_VIETNAMESE
                # vd: term = "2 mũi vaccine cơ", start = 16, end = 20, chữ "2" không có trong dictionary nên sẽ khiến end bị trừ
                # thành 16 (bằng với start), và nếu không có điều kiện kiểm tra "ith_start_term == ith_end_term" thì sẽ khiến
                # start và end trở lại như cũ (16 và 20), vì ở đây gán start = end và đoạn bên dưới gán end = start + LEN_MAX_TERM_VIETNAMESE
                # Nên mới phải cần thêm term không có trong dictionary vào trong danh sách các term của từng văn bản (câu)
                # Và chuyển sang so khớp term kế tiếp
                if ith_start_term == ith_end_term:
                    term = GetTerm(sentence_part_tokenzied, ith_start_term, ith_start_term+1)
                    if ith_sentence not in sentences_tokenized.keys():
                        sentences_tokenized[ith_sentence] = [term]
                    else:
                        sentences_tokenized[ith_sentence].append(term)

                    ith_start_term = ith_end_term + 1
                else:
                    ith_start_term = ith_end_term

                # Nếu vẫn còn nhiều hơn 4 term để xét thì chuyển sang 4 term kế tiếp
                if len(sentence_part_tokenzied) - ith_end_term > LENGTH_MAX_TERM:
                    ith_end_term = ith_start_term + LENGTH_MAX_TERM
                # Nếu chỉ còn nhiều nhất là 4 term thì lấy ra những term cuối cùng của vế để xét
                else:
                    ith_end_term = len(sentence_part_tokenzied)

                term = GetTerm(sentence_part_tokenzied, ith_start_term, ith_end_term)
    return sentences_tokenized.values()

In [301]:
def indexing (lst):
    dic = {}
    i = 0 
    for word in lst:
        n_words = len(word.split(' '))
        if n_words == 1:
            dic[word] = (i,i)
        else: 
            dic[word]= (i,i+n_words-1)
        i += n_words
    return dic 

In [302]:
print(sentences)

['đến nay, lực_lượng chức_năng đã xác_định trên 600 F1 và hàng nghìn F2 liên_quan nhóm bệnh_nhân này.\n', 'dự_báo, số F0 tại xã Hải_Minh dự_kiến sẽ còn gia_tăng trong những ngày tới.\n', 'đến nay, lực_lượng chức_năng đã xác_định trên 600 F1 và hàng nghìn F2 liên_quan nhóm bệnh_nhân này.\n', 'ông Nguyễn_Ngọc_Toàn, tổng_biên_tập báo Thanh_Niên, trao thiết_bị hỗ_trợ học_tập và gửi_gắm nhiều niềm hy_vọng cho các em sẽ gặt_hái được nhiều thành_công học_tập.\n', '24 học_sinh nhận thiết_bị hỗ_trợ từ chương_trình là 24 hoàn_cảnh khó_khăn khác nhau nhưng cùng chung ước_muốn được học trực_tuyến cùng thầy_cô và bạn_bè.\n', 'hay đó là Bùi_Gia_Phúc, học_sinh lớp 4/5 trường tiểu_học Trần_Văn_Ơn, sống với ông_bà nội ngay khi mới chỉ một tháng tuổi.\n', 'thông_qua đó thêm một cơ_hội để các em vượt qua khó_khăn, thử_thách, vươn lên đạt kết_quả tốt nhất trong học_tập, đóng_góp cho tương_lai.\n', 'chương_trình sẽ là dịp để lan tỏa và san_sẻ tình_yêu_thương trong đại_dịch.\n', 'ông Nguyễn_Xuân_Tiến, uỷ_vi

In [303]:
list_words_1 =[]
for e in  range(len(sentences)):
    tmp = PreprocessString(sentences[e])
    for x in range(len(tmp)):
        if '_' in tmp[x]:
            tmp[x] =tmp[x].replace('_',' ')
    list_words_1.append(tmp)


list_words_and_index_1 = []

for e in list_words_1:
     list_words_and_index_1.append(indexing(e))
print(list_words_and_index_1)

    

[{'đến': (0, 0), 'nay': (1, 1), 'lực lượng': (2, 3), 'chức năng': (4, 5), 'đã': (6, 6), 'xác định': (7, 8), 'trên': (9, 9), '600': (10, 10), 'f1': (11, 11), 'và': (12, 12), 'hàng': (13, 13), 'nghìn': (14, 14), 'f2': (15, 15), 'liên quan': (16, 17), 'nhóm': (18, 18), 'bệnh nhân': (19, 20), 'này': (21, 21)}, {'dự báo': (0, 1), 'số': (2, 2), 'f0': (3, 3), 'tại': (4, 4), 'xã': (5, 5), 'hải minh': (6, 7), 'dự kiến': (8, 9), 'sẽ': (10, 10), 'còn': (11, 11), 'gia tăng': (12, 13), 'trong': (14, 14), 'những': (15, 15), 'ngày': (16, 16), 'tới': (17, 17)}, {'đến': (0, 0), 'nay': (1, 1), 'lực lượng': (2, 3), 'chức năng': (4, 5), 'đã': (6, 6), 'xác định': (7, 8), 'trên': (9, 9), '600': (10, 10), 'f1': (11, 11), 'và': (12, 12), 'hàng': (13, 13), 'nghìn': (14, 14), 'f2': (15, 15), 'liên quan': (16, 17), 'nhóm': (18, 18), 'bệnh nhân': (19, 20), 'này': (21, 21)}, {'ông': (0, 0), 'nguyễn ngọc toàn': (1, 3), 'tổng biên tập': (4, 6), 'báo': (7, 7), 'thanh niên': (8, 9), 'trao': (10, 10), 'thiết bị': (11, 

In [304]:
sentences = list()
data = 'tachbangcode.txt'
with open(data, "r", encoding='utf-8') as f:
    content = f.readlines()
    for i in range(len(content)):
        sentence = content[i]
        sentences.append(sentence)
        
print("Number of senteces in data:", len(sentences))
print(sentences)


Number of senteces in data: 40
['đến nay, lực lượng chức năng đã xác định trên 600 F1 và hàng nghìn F2 liên quan nhóm bệnh nhân này.\n', 'dự báo, số F0 tại xã Hải Minh dự kiến sẽ còn gia tăng trong những ngày tới.\n', 'đến nay, lực lượng chức năng đã xác định trên 600 F1 và hàng nghìn F2 liên quan nhóm bệnh nhân này.\n', 'ông Nguyễn Ngọc Toàn, tổng biên tập báo Thanh Niên, trao thiết bị hỗ trợ học tập và gửi gắm nhiều niềm hy vọng cho các em sẽ gặt hái được nhiều thành công học tập.\n', '24 học sinh nhận thiết bị hỗ trợ từ chương trình là 24 hoàn cảnh khó khăn khác nhau nhưng cùng chung ước muốn được học trực tuyến cùng thầy cô và bạn bè.\n', 'hay đó là Bùi Gia Phúc, học sinh lớp 4/5 trường tiểu học Trần Văn Ơn, sống với ông bà nội ngay khi mới chỉ một tháng tuổi.\n', 'thông qua đó thêm một cơ hội để các em vượt qua khó khăn, thử thách, vươn lên đạt kết quả tốt nhất trong học tập, đóng góp cho tương lai.\n', 'chương trình sẽ là dịp để lan tỏa và san sẻ tình yêu thương trong đại dịch.\n

In [305]:
list_words_2 = list(maximumMatching(sentences))

list_words_and_index_2 = []

for e in list_words_2:
     list_words_and_index_2.append(indexing(e))
print(list_words_and_index_2)

[{'đến nay': (0, 1), 'lực lượng': (2, 3), 'chức năng': (4, 5), 'đã': (6, 6), 'xác định': (7, 8), 'trên': (9, 9), '600': (10, 10), 'f1': (11, 11), 'và': (12, 12), 'hàng': (13, 13), 'nghìn': (14, 14), 'f2': (15, 15), 'liên quan': (16, 17), 'nhóm': (18, 18), 'bệnh nhân': (19, 20), 'này': (21, 21)}, {'dự báo': (0, 1), 'số': (2, 2), 'f0': (3, 3), 'tại': (4, 4), 'xã': (5, 5), 'hải': (6, 6), 'minh': (7, 7), 'dự kiến': (8, 9), 'sẽ': (10, 10), 'còn': (11, 11), 'gia tăng': (12, 13), 'trong': (14, 14), 'những': (15, 15), 'ngày': (16, 16), 'tới': (17, 17)}, {'đến nay': (0, 1), 'lực lượng': (2, 3), 'chức năng': (4, 5), 'đã': (6, 6), 'xác định': (7, 8), 'trên': (9, 9), '600': (10, 10), 'f1': (11, 11), 'và': (12, 12), 'hàng': (13, 13), 'nghìn': (14, 14), 'f2': (15, 15), 'liên quan': (16, 17), 'nhóm': (18, 18), 'bệnh nhân': (19, 20), 'này': (21, 21)}, {'ông': (0, 0), 'nguyễn': (1, 1), 'ngọc': (2, 2), 'toàn': (3, 3), 'tổng biên tập': (4, 6), 'báo': (7, 7), 'thanh niên': (8, 9), 'trao': (10, 10), 'thiết

In [306]:
def Evaluate_1_sentences(dic1,dic2):
    recall = 0.0
    precision = 0.0
    matching = 0
    for key1 in dic1:
        for key2 in dic2:
            if key1 == key2 and dic1[key1] == dic2[key2]:
                matching+=1
    precision = matching/len(dic2)
    recall = matching / len(dic1)
    f1 = (precision*recall*2)/(precision+recall)
    return precision, recall,f1
            
        

In [307]:
def Evaluate_all(lst1,lst2):
    precision = []
    recall = []
    f1=[]
    for dic1,dic2 in zip(list_words_and_index_1,list_words_and_index_2):
        a, b,c = Evaluate_1_sentences(dic1,dic2)
        precision.append(a)
        recall.append(b)
        f1.append(c)
    return sum(precision)/len(precision), sum(recall)/len(recall),sum(f1)/len(f1)

In [308]:
accurracy, recall , f1 = Evaluate_all(list_words_1,list_words_2)
print('Accurracy:',accurracy)
print('Recall:',recall)
print('F1:',f1)

Accurracy: 0.8005171280219908
Recall: 0.8464909127997531
F1: 0.8211849199499592
